### Message Passing for REFLACX graph datapoints

In [ ]:
import torch
import dgl

from iou_graph import IOUGraph
from dgl_reflacx_tools.tools import get_node, get_edge, draw
from dgl_reflacx_tools.dgl_reflacx_collection import GraphCollection

In [ ]:
dataset_pth = 'datasets/reflacx_densnet225_iou'

In [ ]:
collection = GraphCollection(dataset_pth, IOUGraph)

In [ ]:
pair = collection.fetch_by_reflacx('1bdf3180-0209f001-967acab6-0b811ea2-3c2e13eb', 'P300R510107')

In [ ]:
g = pair.dgl_graph
labels = pair.dgl_labels
g, labels

In [ ]:
if torch.cuda.device_count() > 0:
    g = g.to(torch.device('cuda:0'))
    labels = labels.to(torch.device('cuda:0'))
g, labels, g.nodes().device, labels.device

In [ ]:
draw(g)